In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score, f1_score, precision_score, recall_score, confusion_matrix
from time import time
import pickle

In [2]:
IoT = pd.read_csv('../../data/merged_data.csv')
# IoT = IoT.iloc[0:15000]

IoT = IoT.dropna()

encoder = LabelEncoder()
IoT['fridge_temp_condition'] = encoder.fit_transform(IoT['fridge_temp_condition'])
IoT['fridge_type'] = encoder.fit_transform(IoT['fridge_type'])
IoT['door_state'] = encoder.fit_transform(IoT['door_state'])
IoT['door_type'] = encoder.fit_transform(IoT['door_type'])
IoT['ml_light_status_x'] = encoder.fit_transform(IoT['ml_light_status_x'])
IoT['ml_type_x'] = encoder.fit_transform(IoT['ml_type_x'])
IoT['gps_type'] = encoder.fit_transform(IoT['gps_type'])
IoT['modbus_type'] = encoder.fit_transform(IoT['modbus_type'])
IoT['ml_light_status_y'] = encoder.fit_transform(IoT['ml_light_status_y'])
IoT['ml_type_y'] = encoder.fit_transform(IoT['ml_type_y'])
IoT['thermostat_type'] = encoder.fit_transform(IoT['thermostat_type'])
IoT['weather_type'] = encoder.fit_transform(IoT['weather_type'])

IoT.head()

,date-time,fridge_temperature,fridge_temp_condition,fridge_label,fridge_type,door_state,door_sphone_signal,door_label,door_type,ml_motion_status_x,...,ml_type_y,thermostat_current_temperature,thermostat_status,thermostat_label,thermostat_type,weather_temperature,weather_pressure,weather_humidity,weather_label,weather_type
0,2019-03-31~12:36:52,13.10,0,0,2,0,False,0,2,1,...,2,25.0,0,0,2,31.788508,1.035,32.036579,0,2
1,2019-03-31~12:36:53,8.65,0,0,2,0,False,0,2,0,...,2,25.0,0,0,2,41.630997,1.035,30.886165,0,2
2,2019-03-31~12:36:54,2.00,1,0,2,0,False,0,2,0,...,2,25.0,0,0,2,42.256959,1.035,19.755908,0,2
3,2019-03-31~12:36:55,4.80,1,0,2,0,False,0,2,1,...,2,25.0,0,0,2,49.116581,1.035,78.949621,0,2
4,2019-03-31~12:36:56,10.70,0,0,2,0,False,0,2,1,...,2,25.0,0,0,2,24.017085,1.035,40.001059,0,2


In [3]:
x = IoT.drop(['modbus_type', 'modbus_label', 'date-time'], axis=1)
y = IoT['modbus_type']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

In [4]:
model4 = LogisticRegression(max_iter=1000)

In [5]:
filename = '../h5s/logistic-regression.h5'
pickle.dump(model4, open(filename, 'wb'))

In [6]:
start = time()
model4.fit(x_train, y_train)
training_time = time() - start

c:\Users\harsh\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [7]:
start = time()
y_pred = model4.predict(x_test)
test_time = time() - start
y_pred

array([2, 2, 2, ..., 2, 2, 2])

In [8]:
# score = model4.score(x_test, y_test)
# print('Test accuracy:', score)

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted', labels=np.unique(y_pred))
recall = recall_score(y_test, y_pred, average='weighted', labels=np.unique(y_pred))
f1 = f1_score(y_test, y_pred, average='weighted', labels=np.unique(y_pred))

print('Accuracy:', accuracy)
print('Precision:', precision)
print('Recall:', recall)
print('F1_Score:', f1)
print(f'Training Time: {training_time}s')
print(f'Testing Time: {test_time}s')
print(classification_report(y_test, y_pred))

Accuracy: 0.8358152686145146
Precision: 0.8201259604925533
Recall: 0.866015625
F1_Score: 0.8141757583220042
Training Time: 14.68832540512085s
Testing Time: 0.006396055221557617s
              precision    recall  f1-score   support

           0       0.76      0.08      0.15       462
           1       0.00      0.00      0.00       169
           2       0.85      0.98      0.91      4454
           3       0.29      0.19      0.23       204
           4       0.00      0.00      0.00        16

    accuracy                           0.84      5305
   macro avg       0.38      0.25      0.26      5305
weighted avg       0.79      0.84      0.79      5305



c:\Users\harsh\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\harsh\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\harsh\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average,

In [9]:
confusion_matrix(y_test, y_pred)

array([[  38,    0,  418,    6,    0],
       [   0,    0,  169,    0,    0],
       [  12,    0, 4358,   84,    0],
       [   0,    0,  166,   38,    0],
       [   0,    0,   11,    5,    0]], dtype=int64)